## Импорт и установка библиотек

In [1]:
# !pip install dbfread # file reader
# !pip install patool # rar unarchiver

In [2]:
import pandas as pd
import patoolib
import os
import numpy as np
import shutil

from dbfread import DBF

In [3]:
def read_dbf(file):
    return pd.DataFrame(iter(DBF(file, encoding='cp866')))

## Разархивирование файлов

## Чтение файлов

### Формат файлов отчета 0409802
- Файл PK802yymm.DBF, где yymm – отчетный год, отчетный месяц: файлы «Консолидированный балансовый отчет»
- Файл PS802yymm.DBF, где yymm – отчетный год, отчетный месяц: файлы «Консолидированный балансовый отчет» раздела «Справочно»

In [4]:
# костыль, чтоб нормально внеслись данные за разные года

df_balance = pd.DataFrame()

for filename in sorted(os.listdir('data/rars/0409802')):
    if not filename.startswith('.'):
        name = filename.split('.')[0]

        patoolib.extract_archive(f'data/rars/0409802/{filename}', outdir=f'data/0409802/{name}', verbosity=-1)
        folder_contents = os.listdir(f'data/0409802/{name}')
        filename = [fname for fname in folder_contents if fname.startswith('PK')][0]
        frame = read_dbf(f'data/0409802/{name}/{filename}')
        
        date = name.split('-')[1]
        
        frame.drop(columns=['KORR_P', 'KORR_M', 'VSEGO'], inplace=True) 

        df_balance = pd.concat([df_balance, frame], join='outer').drop_duplicates()
        shutil.rmtree(f'data/0409802/{name}')

shutil.rmtree(f'data/0409802')
    
df_balance = df_balance.reset_index(drop=True).sort_values(['REGN_GKO', 'STR'])

INFO patool: ... creating output directory `data/0409802/802-20171001'.
INFO patool: ... creating output directory `data/0409802/802-20180101'.
INFO patool: ... creating output directory `data/0409802/802-20180401'.
INFO patool: ... creating output directory `data/0409802/802-20180701'.
INFO patool: ... creating output directory `data/0409802/802-20181001'.
INFO patool: ... creating output directory `data/0409802/802-20190101'.
INFO patool: ... creating output directory `data/0409802/802-20190401'.
INFO patool: ... creating output directory `data/0409802/802-20190701'.
INFO patool: ... creating output directory `data/0409802/802-20191001'.
INFO patool: ... creating output directory `data/0409802/802-20200101'.
INFO patool: ... creating output directory `data/0409802/802-20200401'.
INFO patool: ... creating output directory `data/0409802/802-20200701'.
INFO patool: ... creating output directory `data/0409802/802-20201001'.
INFO patool: ... creating output directory `data/0409802/802-202

In [5]:
for filename in sorted(os.listdir('data/rars/0409802')):
    if not filename.startswith('.'):
        name = filename.split('.')[0]

        patoolib.extract_archive(f'data/rars/0409802/{filename}', outdir=f'data/0409802/{name}', verbosity=-1)
        folder_contents = os.listdir(f'data/0409802/{name}')
        filename = [fname for fname in folder_contents if fname.startswith('PK')][0]
        frame = read_dbf(f'data/0409802/{name}/{filename}')
        
        date = name.split('-')[1]
        frame = frame.rename(columns={
            # если оставлять корректировки, то закомментировать нижнюю строку и раскомментировать верхние
            #'KORR_P': 'KORR_P_'+date,
            #'KORR_M': 'KORR_M_'+date,
            #'VSEGO': 'VSEGO'+date
            'VSEGO': date
        })

        # оставляем только колонку "ВСЕГО", если оставляем корректировки, то закомментировать строку
        frame.drop(columns=['KORR_P', 'KORR_M'], inplace=True) 

        df_balance = df_balance.merge(frame, how='left')
        shutil.rmtree(f'data/0409802/{name}')

shutil.rmtree(f'data/0409802')
    
df_balance = df_balance.reset_index(drop=True)

INFO patool: ... creating output directory `data/0409802/802-20171001'.
INFO patool: ... creating output directory `data/0409802/802-20180101'.
INFO patool: ... creating output directory `data/0409802/802-20180401'.
INFO patool: ... creating output directory `data/0409802/802-20180701'.
INFO patool: ... creating output directory `data/0409802/802-20181001'.
INFO patool: ... creating output directory `data/0409802/802-20190101'.
INFO patool: ... creating output directory `data/0409802/802-20190401'.
INFO patool: ... creating output directory `data/0409802/802-20190701'.
INFO patool: ... creating output directory `data/0409802/802-20191001'.
INFO patool: ... creating output directory `data/0409802/802-20200101'.
INFO patool: ... creating output directory `data/0409802/802-20200401'.
INFO patool: ... creating output directory `data/0409802/802-20200701'.
INFO patool: ... creating output directory `data/0409802/802-20201001'.
INFO patool: ... creating output directory `data/0409802/802-202

In [6]:
df_balance

,REGN_GKO,STR,20171001,20180101,20180401,20180701,20181001,20190101,20190401,20190701,...,20200401,20200701,20201001,20210101,20210401,20210701,20211001,20230701,20231001,20240101
0,1,1,10635086.0,11771672.0,9896604.0,10036567.0,9089270.0,12091355.0,10358528.0,10851933.0,...,19294002.0,17295651.0,16142639.0,14704211.0,13068725.0,12620484.0,10708255.0,21037937.0,28341362.0,21420053.0
1,1,10,23171855.0,26010212.0,23841457.0,24095925.0,23580097.0,25511387.0,7378077.0,7404857.0,...,8346413.0,8059469.0,7826953.0,7487232.0,7494913.0,7260470.0,7082979.0,7096683.0,6468995.0,7003594.0
2,1,10.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,10.2,NaN,NaN,NaN,NaN,NaN,NaN,7378077.0,7404857.0,...,8346413.0,8059469.0,7826953.0,7487232.0,7494913.0,7260470.0,7082979.0,7096683.0,6468995.0,7003594.0
4,1,10.3,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7716,3532,7,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
7717,3532,8,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7718,3532,8.1,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7719,3532,9,NaN,NaN,NaN,NaN,0.0,0.0,41830.0,49611.0,...,67621.0,72820.0,76342.0,74168.0,74002.0,70192.0,66063.0,61088.0,NaN,NaN


In [7]:
df_balance.to_csv('data/data_balances.csv')

### Формат файлов отчета 0409803
- Файл PK803yymm.DBF, где yymm – отчетный год, отчетный месяц: файл «Консолидированный отчет о финансовых результатах»

In [8]:
# костыль, чтоб нормально внеслись данные за разные года

df_pnl = pd.DataFrame()

for filename in sorted(os.listdir('data/rars/0409803')):
    if not filename.startswith('.'):
        name = filename.split('.')[0]

        patoolib.extract_archive(f'data/rars/0409803/{filename}', outdir=f'data/0409803/{name}', verbosity=-1)
        folder_contents = os.listdir(f'data/0409803/{name}')
        filename = [fname for fname in folder_contents if fname.startswith('PK')][0]
        frame = read_dbf(f'data/0409803/{name}/{filename}')
        
        date = name.split('-')[1]

        frame.drop(columns=['KORR_P', 'KORR_M', 'VSEGO'], inplace=True) 

        df_pnl = pd.concat([df_pnl, frame], join='outer').drop_duplicates()
        shutil.rmtree(f'data/0409803/{name}')

shutil.rmtree(f'data/0409803')
    
df_pnl = df_pnl.reset_index(drop=True).sort_values(['REGN_GKO', 'STR'])

INFO patool: ... creating output directory `data/0409803/803-20171001'.
INFO patool: ... creating output directory `data/0409803/803-20180101'.
INFO patool: ... creating output directory `data/0409803/803-20180401'.
INFO patool: ... creating output directory `data/0409803/803-20180701'.
INFO patool: ... creating output directory `data/0409803/803-20181001'.
INFO patool: ... creating output directory `data/0409803/803-20190101'.
INFO patool: ... creating output directory `data/0409803/803-20190401'.
INFO patool: ... creating output directory `data/0409803/803-20190701'.
INFO patool: ... creating output directory `data/0409803/803-20191001'.
INFO patool: ... creating output directory `data/0409803/803-20200101'.
INFO patool: ... creating output directory `data/0409803/803-20200401'.
INFO patool: ... creating output directory `data/0409803/803-20200701'.
INFO patool: ... creating output directory `data/0409803/803-20201001'.
INFO patool: ... creating output directory `data/0409803/803-202

In [9]:
for filename in sorted(os.listdir('data/rars/0409803')):
    if not filename.startswith('.'):
        name = filename.split('.')[0]

        patoolib.extract_archive(f'data/rars/0409803/{filename}', outdir=f'data/0409803/{name}', verbosity=-1)
        folder_contents = os.listdir(f'data/0409803/{name}')
        filename = [fname for fname in folder_contents if fname.startswith('PK')][0]
        frame = read_dbf(f'data/0409803/{name}/{filename}')
        
        date = name.split('-')[1]
        frame = frame.rename(columns={
            # если оставлять корректировки, то закомментировать нижнюю строку и раскомментировать верхние
            #'KORR_P': 'KORR_P_'+date,
            #'KORR_M': 'KORR_M_'+date,
            #'VSEGO': 'VSEGO'+date
            'VSEGO': date
        })

        # оставляем только колонку "ВСЕГО", если оставляем корректировки, то закомментировать строку
        frame.drop(columns=['KORR_P', 'KORR_M'], inplace=True) 

        df_pnl = df_pnl.merge(frame, how='left')
        shutil.rmtree(f'data/0409803/{name}')

shutil.rmtree(f'data/0409803')
    
df_pnl = df_pnl.reset_index(drop=True)

INFO patool: ... creating output directory `data/0409803/803-20171001'.
INFO patool: ... creating output directory `data/0409803/803-20180101'.
INFO patool: ... creating output directory `data/0409803/803-20180401'.
INFO patool: ... creating output directory `data/0409803/803-20180701'.
INFO patool: ... creating output directory `data/0409803/803-20181001'.
INFO patool: ... creating output directory `data/0409803/803-20190101'.
INFO patool: ... creating output directory `data/0409803/803-20190401'.
INFO patool: ... creating output directory `data/0409803/803-20190701'.
INFO patool: ... creating output directory `data/0409803/803-20191001'.
INFO patool: ... creating output directory `data/0409803/803-20200101'.
INFO patool: ... creating output directory `data/0409803/803-20200401'.
INFO patool: ... creating output directory `data/0409803/803-20200701'.
INFO patool: ... creating output directory `data/0409803/803-20201001'.
INFO patool: ... creating output directory `data/0409803/803-202

In [10]:
df_pnl.to_csv('data/data_pnls.csv')

### Формат файлов отчета 0409135
- Файл mmyyyy_135_3.DBF, где yymm – отчетный год, отчетный месяц: файл с обязательными нормативами
- https://www.cbr.ru/vfs/credit/formats/135-20210501.pdf

In [11]:
# костыль, чтоб нормально внеслись данные за разные года

df_norms = pd.DataFrame()

for filename in sorted(os.listdir('data/rars/135')):
    if not filename.startswith('.'):
        name = filename.split('.')[0]

        patoolib.extract_archive(f'data/rars/135/{filename}', outdir=f'data/135/{name}', verbosity=-1)
        folder_contents = os.listdir(f'data/135/{name}')
        filename = [fname for fname in folder_contents if fname.upper().endswith('_135_3.DBF')][0]
        print(filename)
        frame = read_dbf(f'data/135/{name}/{filename}')
        
        date = name.split('-')[1]
        frame = frame.rename(columns={
            'REGN': 'REGN_GKO', 
            'C1_3': 'NAME_NORM'
        })
        
        frame.drop(columns=['C2_3', 'C3_3', 'C4_3'], inplace=True) 

        df_norms = pd.concat([df_norms, frame], join='outer').drop_duplicates()
        shutil.rmtree(f'data/135/{name}')

shutil.rmtree(f'data/135')
    
df_norms = df_norms.reset_index(drop=True).sort_values(['REGN_GKO', 'NAME_NORM'])

INFO patool: ... creating output directory `data/135/135-20171001'.
INFO patool: ... creating output directory `data/135/135-20180101'.
INFO patool: ... creating output directory `data/135/135-20180401'.
INFO patool: ... creating output directory `data/135/135-20180701'.


092017_135_3.dbf
122017_135_3.dbf
032018_135_3.dbf


INFO patool: ... creating output directory `data/135/135-20181001'.
INFO patool: ... creating output directory `data/135/135-20190101'.
INFO patool: ... creating output directory `data/135/135-20190401'.
INFO patool: ... creating output directory `data/135/135-20190701'.
INFO patool: ... creating output directory `data/135/135-20191001'.


062018_135_3.dbf
092018_135_3.dbf
122018_135_3.dbf
032019_135_3.dbf
062019_135_3.dbf
092019_135_3.dbf


INFO patool: ... creating output directory `data/135/135-20200101'.
INFO patool: ... creating output directory `data/135/135-20200401'.
INFO patool: ... creating output directory `data/135/135-20200701'.
INFO patool: ... creating output directory `data/135/135-20201001'.
INFO patool: ... creating output directory `data/135/135-20210101'.
INFO patool: ... creating output directory `data/135/135-20210401'.


122019_135_3.dbf
032020_135_3.dbf
062020_135_3.DBF
092020_135_3.DBF
122020_135_3.DBF
032021_135_3.DBF


INFO patool: ... creating output directory `data/135/135-20210701'.
INFO patool: ... creating output directory `data/135/135-20211001'.
INFO patool: ... creating output directory `data/135/135-20220101'.
INFO patool: ... creating output directory `data/135/135-20230701'.
INFO patool: ... creating output directory `data/135/135-20231001'.
INFO patool: ... creating output directory `data/135/135-20240101'.


062021_135_3.DBF
092021_135_3.DBF
122021_135_3.DBF
062023_135_3.dbf
092023_135_3.dbf
122023_135_3.dbf


In [12]:
for filename in sorted(os.listdir('data/rars/135')):
    if not filename.startswith('.'):
        name = filename.split('.')[0]

        patoolib.extract_archive(f'data/rars/135/{filename}', outdir=f'data/135/{name}', verbosity=-1)
        folder_contents = os.listdir(f'data/135/{name}')
        filename = [fname for fname in folder_contents if fname.upper().endswith('_135_3.DBF')][0]
        frame = read_dbf(f'data/135/{name}/{filename}')
        
        date = name.split('-')[1]
        frame = frame.rename(columns={
            'REGN': 'REGN_GKO', 
            'C2_3': date
        })

        # оставляем только колонку со значениями
        frame.drop(columns=['C3_3', 'C4_3'], inplace=True) 

        df_norms = df_norms.merge(frame, how='left')
        shutil.rmtree(f'data/135/{name}')

shutil.rmtree(f'data/135')
    
df_norms = df_norms.reset_index(drop=True)

INFO patool: ... creating output directory `data/135/135-20171001'.
INFO patool: ... creating output directory `data/135/135-20180101'.
INFO patool: ... creating output directory `data/135/135-20180401'.
INFO patool: ... creating output directory `data/135/135-20180701'.
INFO patool: ... creating output directory `data/135/135-20181001'.
INFO patool: ... creating output directory `data/135/135-20190101'.
INFO patool: ... creating output directory `data/135/135-20190401'.
INFO patool: ... creating output directory `data/135/135-20190701'.
INFO patool: ... creating output directory `data/135/135-20191001'.
INFO patool: ... creating output directory `data/135/135-20200101'.
INFO patool: ... creating output directory `data/135/135-20200401'.
INFO patool: ... creating output directory `data/135/135-20200701'.
INFO patool: ... creating output directory `data/135/135-20201001'.
INFO patool: ... creating output directory `data/135/135-20210101'.
INFO patool: ... creating output directory `data

In [13]:
df_norms.head()

,REGN_GKO,NAME_NORM,C1_3,20171001,20180101,20180401,20180701,20181001,20190101,20190401,...,20200701,20201001,20210101,20210401,20210701,20211001,20220101,20230701,20231001,20240101
0,1,Н1.0,Н1.0,18.192,18.157,18.617,18.057,17.733,16.149,16.465,...,16.971,16.739,18.894,18.100,17.659,19.070,19.446,40.836,39.249,46.328
1,1,Н1.0,Н1.1,14.537,14.114,15.171,14.246,14.386,12.621,13.597,...,15.677,15.754,16.892,17.181,16.317,17.978,18.139,37.979,36.044,39.475
2,1,Н1.0,Н1.2,14.537,14.114,15.171,14.246,14.386,12.621,13.597,...,15.677,15.754,16.892,17.181,16.317,17.978,18.139,37.979,36.044,39.475
3,1,Н1.0,Н2,126.360,117.094,129.566,197.754,160.969,191.101,147.949,...,226.102,158.813,180.433,154.870,131.588,133.302,236.753,156.600,174.942,231.281
4,1,Н1.0,Н3,278.646,224.919,156.418,187.032,199.355,222.584,252.017,...,268.706,340.291,447.926,543.247,481.415,447.895,331.185,338.194,408.334,1391.507


In [14]:
df_norms.to_csv('data/data_norms.csv')

### Формат файлов "Консолидированная финансовая отчетность"
!!! по тому какая форма какой файл я не уверена 

- Файл F815yymm.DBF, где yymm – отчетный год, отчетный месяц: ОФП **=> дополнение в `df_balance`**
- Файл F816yymm.DBF, где yymm – отчетный год, отчетный месяц: ОПУ **=> дополнение в `df_pnl`**
- Файл F817yymm.DBF, где yymm – отчетный год, отчетный месяц: ОДДС

In [ ]:
# костыль, чтоб нормально внеслись данные за разные года

df_pnl_early = pd.DataFrame()

for filename in sorted(os.listdir('data/rars/Консолидированная отчетность')):
    if not filename.startswith('.'):
        name = filename.split('.')[0]

        #patoolib.extract_archive(f'data/rars/Консолидированная отчетность/{filename}', outdir=f'data/Консолидированная отчетность/{name}', verbosity=-1)
        folder_contents = os.listdir(f'data/rars/Консолидированная отчетность/{name}')
        filename = [fname for fname in folder_contents if fname.startswith('F816')][0]
        frame = read_dbf(f'data/rars/Консолидированная отчетность/{name}/{filename}')
        
        date = name.split('-')[1]

        frame.drop(columns=['CP', 'NUM_PRIM', 'S_LAST'], inplace=True)

        df_pnl_early = pd.concat([df_pnl_early, frame], join='outer').drop_duplicates()
    
#df_pnl_early = df_pnl_early.reset_index(drop=True).sort_values(['REGN_GKO', 'STR'])

In [ ]:
df_pnl_early.head()

## Обработка показателей

Рассматриваемые показатели:

1) Отношение собственных средств (34) к чистым активам (34-21)
2) Отношение резервов (2.1.1 - обязательные, 20 - возможные потери, 31.1 - оценочные под ожидаемые кредитные убытки) к чистым активам (34-21)
3) Чистые активы (34-21) или их логарифм
4) _Отношение просроченной задолженности по ссудам к кредитам экономики_
5) _Отношение просроченной задолженности по ссудам к обязательным резервам в ЦБ РФ_
6) _Отношение оборотов по корреспондентским счетам за период времени к чистым активам_
7) Отношение балансовой прибыли (32) к чистым активам (34-21)
8) Отношение чистой прибыли (32.1) к чистым активам (34-21)
9) Норматив текущей ликвидности
10) _Отношение ликвидных активов к чистым активам_
11) _Отношение депозитов физических лиц до 30 дней к депозитам физических лиц_
12) _Отношение объема негосударственных ценных бумаг к чистым активам_

Справка по строкам: https://www.consultant.ru/document/cons_doc_LAW_444612/c1c3277496f8d7c26caf7152797c75ab73d1a385/

In [15]:
def choose_str(df, str, str_col='STR', regn_col='REGN_GKO'):
    return df[df[str_col] == str].drop(columns=str_col).set_index(regn_col)

In [16]:
net_income = choose_str(df_balance, '32.1') # чистая прибыль [прибыль (убыток) за отчетный период]
balance_income = choose_str(df_balance, '32') # балансовая прибыль [Прибыль (убыток)]

capital = choose_str(df_balance, '34') # собственные средства
reserves = choose_str(df_balance, '20') + choose_str(df_balance, '31.1') + choose_str(df_balance, '2.1.1')

liabilities = choose_str(df_balance, '21') # обязательства

net_assets = capital - liabilities # Активы-нетто (чистые активы) - стоимость капитала по рыночной цене минус долговые обязательства.
liquidity_norm = choose_str(df_norms, 'H3.0', str_col='NAME_NORM')

In [17]:
X1 = capital / net_assets
X2 = reserves / net_assets
X3 = np.log(net_assets) # логарифм
# X4
# X5
# X6
X7 = balance_income / net_assets
X8 = net_income / net_assets
X9 = liquidity_norm
# X10
# X11
# X12

/opt/anaconda3/lib/python3.10/site-packages/pandas/core/internals/blocks.py:351: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/opt/anaconda3/lib/python3.10/site-packages/pandas/core/internals/blocks.py:351: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)


In [18]:
# regressors = pd.DataFrame(df_balance['REGN_GKO'].unique().copy(), columns=['REGN_GKO'])